In [1]:
import pandas as pd
from konlpy.tag import Okt
import json

In [2]:
# Import Dataset
df = pd.read_csv('data/단양구경시장 유리네 수제 궁채장아찌 절임 1KG [원산지상세설명에 표시]_500.csv', encoding='utf-8-sig')

# 토큰화
okt = Okt()
words = []
for review in df['reviews']:
    word = okt.morphs(str(review))
    words.append(word)

# 토큰화 결과 저장
df['Okt_tokens'] = words
print(df[['reviews', 'Okt_tokens']][:4])

                                             reviews  \
0                                    항상 맛있게 먹고 있습니다.   
1                너무 맛나요 ~ 짜지도 않고 적당해요~~ 다음에 또 구매할게용~   
2  지인추천으로 사먹고 맛있어서 두통재주문했어요. 집에서 고기먹을때 없으면 서운한 반찬...   
3  흠..저희 장아찌 좋아해요..근데 궁채에 고구마에서 나오는 길다린 씽?이라고해야하나...   

                                          Okt_tokens  
0                             [항상, 맛있게, 먹고, 있습니다, .]  
1  [너무, 맛, 나, 요, ~, 짜지도, 않고, 적당해요, ~~, 다음, 에, 또, ...  
2  [지인, 추천, 으로, 사먹고, 맛있어서, 두통, 재, 주문, 했어요, ., 집, ...  
3  [흠, .., 저희, 장아찌, 좋아해요, .., 근데, 궁채, 에, 고구마, 에서,...  


In [3]:
# Import Dictionary
with open('data/SentiWord_info.json', encoding = 'utf-8-sig', mode = 'r') as f: 
  json_data = json.load(f)

Dict = pd.DataFrame(json_data)
print(Dict.head(4))

    word word_root polarity
0    (-;         (        1
1  (;_;)     (;_;)       -1
2   (^^)      (^^)        1
3  (^-^)     (^-^)        1


In [12]:
print(Dict['polarity'].value_counts())
# -2 매우 부정, -1 부정, 0 중립, 1 긍정, 2 매우 긍정

polarity
-1    5030
-2    4799
2     2602
1     2269
0      154
Name: count, dtype: int64


In [5]:
# 감성점수 매기기
def scoring(tokens):
    total_score = 0

    for token in tokens:
        if Dict['word'].isin([token]).any():   # 단어 탐색
            score = Dict.loc[Dict['word'] == token, 'polarity']   # 각 단어의 극성 점수
            score = int(score)
            total_score += score

    return total_score

In [ ]:
result = []
for review_tokens in df['Okt_tokens']:
    score = scoring(review_tokens)
    result.append(score)

df['KNU_Score'] = result

In [14]:
df[['reviews','KNU_Score']]
# print(df[['reviews','KNU_Score']])

C:\Users\njh26\AppData\Local\Temp\ipykernel_1900\3634431190.py:7: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  score = int(score)


,reviews,KNU_Score
0,항상 맛있게 먹고 있습니다.,2
1,너무 맛나요 ~ 짜지도 않고 적당해요~~ 다음에 또 구매할게용~,0
2,지인추천으로 사먹고 맛있어서 두통재주문했어요. 집에서 고기먹을때 없으면 서운한 반찬...,-4
3,흠..저희 장아찌 좋아해요..근데 궁채에 고구마에서 나오는 길다린 씽?이라고해야하나...,-2
4,아삭아삭 너무 짜지도 달지도 않고 맛있네요,0
...,...,...
495,믿고 먹는 장아찌~~ 짜지 않고 아삭아삭 너무 맛나요^^,2
496,잘받았습니다 맛있어보여요,1
497,당일에 만들어서 보내주신 것 같아요.\n주문 일주일만에 반 먹었어요!,0
498,두번째 주문하고\n아버지께서 좋아히십니다,0


In [ ]:
# 점수 -> 긍/부정 판단으로 변환
def pos_neg(num):
    if num > 0:
        return '긍정'
    elif num < 0:
        return '부정'
    else:
        return '중립'

In [9]:
df['KNU_label'] = df['KNU_Score'].apply(pos_neg)
print(df['KNU_label'].value_counts())

KNU_label
중립    263
긍정    202
부정     35
Name: count, dtype: int64


In [13]:
df[['reviews','KNU_label']]
# print(df[['reviews','KNU_label']])

,reviews,KNU_label
0,항상 맛있게 먹고 있습니다.,긍정
1,너무 맛나요 ~ 짜지도 않고 적당해요~~ 다음에 또 구매할게용~,중립
2,지인추천으로 사먹고 맛있어서 두통재주문했어요. 집에서 고기먹을때 없으면 서운한 반찬...,부정
3,흠..저희 장아찌 좋아해요..근데 궁채에 고구마에서 나오는 길다린 씽?이라고해야하나...,부정
4,아삭아삭 너무 짜지도 달지도 않고 맛있네요,중립
...,...,...
495,믿고 먹는 장아찌~~ 짜지 않고 아삭아삭 너무 맛나요^^,긍정
496,잘받았습니다 맛있어보여요,긍정
497,당일에 만들어서 보내주신 것 같아요.\n주문 일주일만에 반 먹었어요!,중립
498,두번째 주문하고\n아버지께서 좋아히십니다,중립


In [ ]:
# 결과 저장
df.to_csv('KNU_Sentimental.csv', encoding = 'utf-8-sig', index = False)